##### Import the Packages

In [ ]:
import os
from dotenv import load_dotenv

from huggingface_hub import login

In [ ]:
from AdversarialPromptGenerator import AdversarialPromptGenerator

from TokenSHAP.token_shap.base import LocalModel, HuggingFaceEmbeddings
from TokenSHAP.token_shap.token_shap import TokenizerSplitter, TokenSHAP

##### Retrieve the Hugging Face Key

In [ ]:
load_dotenv()
hf_api_key = os.getenv("HUGGINGFACE_API_KEY")
if not hf_api_key:
    raise RuntimeError("Missing HUGGINGFACE_API_KEY. Set it in your environment or .env file.")
login(hf_api_key)

##### Instantiate TokenSHAP

In [ ]:
model_path = "meta-llama/Llama-3.2-1B-Instruct"
local_model = LocalModel(model_name=model_path, max_new_tokens=10)
hf_embedding = HuggingFaceEmbeddings()
splitter = TokenizerSplitter(local_model.tokenizer)
token_shap_local = TokenSHAP(model=local_model, splitter=splitter, vectorizer=hf_embedding, debug=True)

##### Instantiate PromptGenerator

In [ ]:
adv_prompt_generator = AdversarialPromptGenerator()
adversarial_suffix_path = "./adv_suffixes.pt" # tensor of all 100 suffixes
all_prompts = adv_prompt_generator.get_from(adversarial_suffix_path)

In [ ]:
for prompt in all_prompts:
    df_local = token_shap_local.analyze(prompt, sampling_ratio=0.1)
    token_shap_local.print_colored_text()